In [5]:
import pandas as pd
import unicodedata
import requests
from datetime import datetime as dt

from bs4 import BeautifulSoup as soup
from helpers import regression as rh
from helpers.file import load_from_file
from services import MySQLService

In [24]:
season = 2024
target_stat = "ast"

mysql = MySQLService()

In [25]:
model_info = load_from_file("../dist/betting", target_stat)

print("Model name:", model_info["name"])
print("Model features:", model_info["features"])
print("Model stats_columns:", model_info["stats_columns"])
print("Model confidence margin:", model_info["confidence_margin"])
print("Model standard deviation:", model_info["std"])

Model name: GradientBoosting
Model features: ['ast_avg', 'opponent_avg_conceded_ast', 'days_since_last_game', 'tov_avg', 'opponent_avg_conceded_tov']
Model stats_columns: ['pts', 'fgm', 'fga', 'fg_pct', 'fg3m', 'fg3a', 'fg3_pct', 'ftm', 'fta', 'ft_pct', 'oreb', 'dreb', 'reb', 'ast', 'stl', 'blk', 'tov', 'pf']
Model confidence margin: 1.3799492945349403
Model standard deviation: 1.873466285270969


In [26]:
def remove_accents(input_str):
    return ''.join(
        c for c in unicodedata.normalize('NFD', input_str)
        if unicodedata.category(c) != 'Mn'
    )

season_games_plyrs = mysql.execute_query(f"""
    SELECT g.id as game_id, g.date, g.season, g.is_playoff, g.winner, g.home_id, p.name as player_name, g.away_id, pg.team_id, pg.player_id, pg.minutes, pg.pts, pg.fgm, pg.fga, pg.fg_pct, pg.fg3m, pg.fg3a, pg.fg3_pct, pg.ftm, pg.fta, pg.ft_pct, pg.oreb, pg.dreb, pg.reb, pg.ast, pg.stl, pg.blk, pg.tov, pg.pf, pg.plus_minus
    FROM player_games AS pg
        LEFT JOIN games as g on pg.game_id = g.id 
        LEFT JOIN players as p on pg.player_id = p.id
    WHERE g.season = 2024
    ORDER BY g.date ASC""")

teams_df = mysql.get_data("teams")

order_by_clause = "date ASC"
games_df = mysql.get_data("games", order_by_clause=order_by_clause)

stats_columns = ['pts','fgm', 'fga', 'fg_pct', 'fg3m', 'fg3a', 'fg3_pct', 'ftm', 'fta',
    'ft_pct', 'oreb', 'dreb', 'reb', 'ast', 'stl', 'blk', 'tov', 'pf']

rh.build_avg_columns(season_games_plyrs, stats_columns)

rh.build_days_since_last_game_col(season_games_plyrs)

season_games_plyrs = rh.build_opponent_conceded_columns(season_games_plyrs, games_df, stats_columns)

season_games_plyrs["player_name"] = season_games_plyrs["player_name"].str.replace(" Jr.", "")
season_games_plyrs['player_name'] = season_games_plyrs['player_name'].apply(remove_accents)

display(season_games_plyrs)

c:\Users\caiog\Projects\Personal\NBA-Predictor\src\services\mysql.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  resp = pd.read_sql_query(query, db)
100%|██████████| 18/18 [00:04<00:00,  4.21it/s]


,game_id,date,season,is_playoff,winner,home_id,player_name,away_id,team_id_x,player_id,...,opponent_avg_conceded_fta,opponent_avg_conceded_ft_pct,opponent_avg_conceded_oreb,opponent_avg_conceded_dreb,opponent_avg_conceded_reb,opponent_avg_conceded_ast,opponent_avg_conceded_stl,opponent_avg_conceded_blk,opponent_avg_conceded_tov,opponent_avg_conceded_pf
0,22400061,2024-10-22,2024,0,H,1610612738,Jordan Walsh,1610612752,1610612738,1641775,...,18.4,0.7741,10.6,29.7,40.3,28.1,6.0,6.1,12.0,21.5
1,22400062,2024-10-22,2024,0,H,1610612747,Dalton Knecht,1610612750,1610612747,1642261,...,19.7,0.8202,9.0,31.3,40.3,22.7,5.9,5.6,12.1,19.5
2,22400061,2024-10-22,2024,0,H,1610612738,Tyler Kolek,1610612752,1610612752,1642278,...,16.2,0.7482,9.6,31.9,41.5,22.6,5.6,2.4,13.0,18.9
3,22400062,2024-10-22,2024,0,H,1610612747,Bronny James,1610612750,1610612747,1642355,...,19.7,0.8202,9.0,31.3,40.3,22.7,5.9,5.6,12.1,19.5
4,22400061,2024-10-22,2024,0,H,1610612738,Pacome Dadiet,1610612752,1610612752,1642359,...,16.2,0.7482,9.6,31.9,41.5,22.6,5.6,2.4,13.0,18.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8274,22401216,2024-12-15,2024,0,H,1610612754,Yves Missi,1610612740,1610612740,1642274,...,23.0,0.7328,9.0,33.2,42.2,28.8,8.6,4.7,16.4,18.2
8275,22401216,2024-12-15,2024,0,H,1610612754,Johnny Furphy,1610612740,1610612754,1642277,...,21.7,0.7634,12.5,37.8,50.3,30.1,9.9,4.8,16.0,17.8
8276,22401219,2024-12-15,2024,0,H,1610612756,Oso Ighodaro,1610612757,1610612756,1642345,...,25.1,0.7902,12.4,32.8,45.2,28.8,9.2,6.9,14.5,17.8
8277,22401219,2024-12-15,2024,0,H,1610612756,Ryan Dunn,1610612757,1610612756,1642346,...,25.1,0.7902,12.4,32.8,45.2,28.8,9.2,6.9,14.5,17.8


In [27]:
season_games_plyrs.columns

Index(['game_id', 'date', 'season', 'is_playoff', 'winner', 'home_id',
       'player_name', 'away_id', 'team_id_x', 'player_id', 'minutes', 'pts',
       'fgm', 'fga', 'fg_pct', 'fg3m', 'fg3a', 'fg3_pct', 'ftm', 'fta',
       'ft_pct', 'oreb', 'dreb', 'reb', 'ast', 'stl', 'blk', 'tov', 'pf',
       'plus_minus', 'pts_avg', 'fgm_avg', 'fga_avg', 'fg_pct_avg', 'fg3m_avg',
       'fg3a_avg', 'fg3_pct_avg', 'ftm_avg', 'fta_avg', 'ft_pct_avg',
       'oreb_avg', 'dreb_avg', 'reb_avg', 'ast_avg', 'stl_avg', 'blk_avg',
       'tov_avg', 'pf_avg', 'days_since_last_game', 'opponent_team_id',
       'team_id_y', 'opponent_avg_conceded_pts', 'opponent_avg_conceded_fgm',
       'opponent_avg_conceded_fga', 'opponent_avg_conceded_fg_pct',
       'opponent_avg_conceded_fg3m', 'opponent_avg_conceded_fg3a',
       'opponent_avg_conceded_fg3_pct', 'opponent_avg_conceded_ftm',
       'opponent_avg_conceded_fta', 'opponent_avg_conceded_ft_pct',
       'opponent_avg_conceded_oreb', 'opponent_avg_conceded

In [28]:
def get_games_info():
    url = "https://www.rotowire.com/basketball/nba-lineups.php"
    result = requests.get(url)
    doc = soup(result.text, "html.parser")
    games = doc.find_all("div", {"class": "lineup__box"})
    
    for g in games:
        try:
            game_dict = {
                'season': season,
                'date': dt.now()
            }
            
            teams = [x.text for x in g.find_all("div", {"class": "lineup__abbr"})]
            if len(teams) == 0:
                continue

            game_dict["away_abbv"] = teams[0]
            game_dict["home_abbv"] = teams[1]

            home_team = teams_df.loc[teams_df['abbreviation'] == game_dict["home_abbv"]].iloc[0]
            away_team = teams_df.loc[teams_df['abbreviation'] == game_dict["away_abbv"]].iloc[0]
            
            game_dict["away_id"] = away_team["id"]
            game_dict["home_id"] = home_team["id"]

            away_lineup = g.find("ul", {"class": "lineup__list is-visit"})
            home_lineup = g.find("ul", {"class": "lineup__list is-home"})

            away_lineup = [x.text.split('\n')[-2] for x in away_lineup.find_all("li", {"class": "lineup__player"})][:5]
            home_lineup = [x.text.split('\n')[-2] for x in home_lineup.find_all("li", {"class": "lineup__player"})][:5]

            home_players_df = rh.build_next_games_df(home_lineup, home_team, away_team, season_games_plyrs, stats_columns, games_df)
            away_players_df = rh.build_next_games_df(away_lineup, away_team, home_team, season_games_plyrs, stats_columns, games_df)

            merged_df = pd.concat([home_players_df, away_players_df], ignore_index=True)

            merged_df['pred'] = model_info["model"].predict(merged_df[model_info["features"]])

            merged_df['lower_bound'] = (merged_df['pred'] - model_info["std"]).round()
            merged_df['upper_bound'] = (merged_df['pred'] + model_info["std"]).round()
            
            display(merged_df[["lineup_name", 'player_name', 'team_name', 'opp_name', 'pred', 'lower_bound', 'upper_bound']])
        except Exception as e:
            print(e)
            continue

In [29]:
get_games_info()

,lineup_name,player_name,team_name,opp_name,pred,lower_bound,upper_bound
0,S. Gilgeous-Alexander,Shai Gilgeous-Alexander,Oklahoma City Thunder,Milwaukee Bucks,5.703951,4.0,8.0
1,Isaiah Joe,Isaiah Joe,Oklahoma City Thunder,Milwaukee Bucks,1.694015,-0.0,4.0
2,Luguentz Dort,Luguentz Dort,Oklahoma City Thunder,Milwaukee Bucks,1.697263,-0.0,4.0
3,J. Williams,Jalen Williams,Oklahoma City Thunder,Milwaukee Bucks,5.024588,3.0,7.0
4,I. Hartenstein,Isaiah Hartenstein,Oklahoma City Thunder,Milwaukee Bucks,3.948719,2.0,6.0
5,D. Lillard,Damian Lillard,Milwaukee Bucks,Oklahoma City Thunder,7.164191,5.0,9.0
6,Andre Jackson,Andre Jackson,Milwaukee Bucks,Oklahoma City Thunder,1.506194,-0.0,3.0
7,T. Prince,Taurean Prince,Milwaukee Bucks,Oklahoma City Thunder,1.342559,-1.0,3.0
8,G. Antetokounmpo,Giannis Antetokounmpo,Milwaukee Bucks,Oklahoma City Thunder,6.192625,4.0,8.0
9,Brook Lopez,Brook Lopez,Milwaukee Bucks,Oklahoma City Thunder,1.445226,-0.0,3.0
